In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH1 = './archive/data-final-clean.csv'

In [3]:
df1 = pd.read_csv(PATH1)

In [4]:
sample_df = df1.sample(n=11000, random_state=42)
est_columns = [col for col in sample_df.columns if col.startswith('EST')]
csn_columns = [col for col in sample_df.columns if col.startswith('CSN')]
est_responses_matrix = sample_df[est_columns].values
csn_responses_matrix = sample_df[csn_columns].values
min_agreement_percentage = 0.6
est_similarity_matrix = (est_responses_matrix[:, np.newaxis, :] == est_responses_matrix).mean(axis=2)
csn_similarity_matrix = (csn_responses_matrix[:, np.newaxis, :] == csn_responses_matrix).mean(axis=2)
average_similarity_matrix = (est_similarity_matrix + csn_similarity_matrix) / 2
np.fill_diagonal(average_similarity_matrix, 0)
connections = np.argwhere(average_similarity_matrix >= min_agreement_percentage)
connections_df = pd.DataFrame(connections, columns=['node_1', 'node_2'])
print(connections_df)

        node_1  node_2
0            0      96
1            0     116
2            0     573
3            0     623
4            0    1950
...        ...     ...
623683   10999     349
623684   10999    3280
623685   10999    4187
623686   10999    4274
623687   10999    9279

[623688 rows x 2 columns]


In [5]:
caminho_do_arquivo_csv = 'big_five_dataset.csv'
connections_df.to_csv(caminho_do_arquivo_csv, index=False)

In [6]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw

In [7]:
PATH = './big_five_dataset.csv'

In [8]:
g = gte.Graph(directed=False)

In [9]:
g = gte.Graph(directed=False)

In [10]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

In [11]:
def get_or_add_edge(g, node1, node2):
    e = g.edge_by_ids(node1, node2)
    if e is None:
        e = g.add_edge_by_ids(node1, node2)
    return e

In [12]:
with open(PATH) as file:
    next(file)
    for line in file:
        line = line.strip().split(',')
        #Cria ou pega os vértices
        get_or_add_vertex(g,line[0])
        get_or_add_vertex(g,line[1])
        #Cria ou pega as areatas
        get_or_add_edge(g,line[0],line[1])

In [13]:
g = gte.clean(g)

In [14]:
layout = draw.sfdp_layout(g)

In [15]:
gte.move(g, layout)

In [16]:
gte.save(g, 'big_five_agree.net.gz')

In [17]:
r = netpixi.render('big_five_agree.net.gz',infinite=True)

In [18]:
r.vertex_default(size=4, bwidth=1)

In [19]:
r.edge_default(width=1)